In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Peak into the dataset files 

In [ ]:

import pandas as pd
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_items.head(5)

In [ ]:
df_test= pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df_test.head(5)

In [ ]:
df_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_train.head(5)

In [ ]:
df_item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_item_categories.head(5)

## Feature engineering

In [ ]:
df = df_train
df = df.groupby(['date_block_num','shop_id','item_id']).agg(
    {'item_price':'mean',
     'item_cnt_day':'sum'}).reset_index()
df.head(5)

In [ ]:
df_test['date_block_num']=34
item_price = dict(df.groupby('item_id')['item_price'].last().reset_index().values)
df_test['item_price'] = df_test['item_id'].map(item_price)
df_test['item_price'] = df_test['item_price'].fillna(df_test['item_price'].median())
df_test.head(5)

In [ ]:
# df.head(5)
df =  df.sample(frac = 1)

# Model training

In [ ]:
np.array(df.drop(['item_cnt_day'],1)) #.values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
x = np.array(df.drop(['item_cnt_day'],1))
y = np.array(df.iloc[:,4])
SC = MinMaxScaler()
SC.fit(x)
X = SC.transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X,y,
                                               test_size = 0.25, random_state = 33)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error



## RMSE root mean squared error

RMSE is a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
def rmse(ytrain, train_pred, ytest, test_pred):
    print('RMSE Train data', mean_squared_error(ytrain, train_pred))
    print('RMSE Test data', mean_squared_error(ytest, test_pred))

In [ ]:
X_submission = SC.fit_transform(df_test.values)

*RandomForestRegressor*

In [ ]:
# rf = RandomForestRegressor()
# rf.fit(xtrain, ytrain)
# rf_tr_pred = rf.predict(xtrain)
# rf_te_pred = rf.predict(xtest)

# rmse(ytrain,rf_tr_pred, ytest,rf_te_pred)


*KNeighborsRegressor*

In [ ]:
kn = KNeighborsRegressor()
kn.fit(xtrain, ytrain)
kn_tr_pred = kn.predict(xtrain)
kn_te_pred = kn.predict(xtest)
# kn.predict(X_submission)
rmse(ytrain,kn_tr_pred, ytest,kn_te_pred)

*LinearRegression*

In [ ]:
lr = LinearRegression()
lr.fit(xtrain, ytrain)
lr_tr_pred = lr.predict(xtrain)
lr_te_pred = lr.predict(xtest)
# lr.predict(X_submission)
rmse(ytrain,lr_tr_pred, ytest,lr_te_pred)

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures  
# poly_regs= PolynomialFeatures(degree= 4)  
# x_poly= poly_regs.fit_transform(X)  
# # kn.fit(x_poly, y)
# clf = LinearRegression()
# clf.fit(x_poly, y)
# poly_pred = clf.predict()  
# print(poly_pred) 

In [ ]:
df1 = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df2 = pd.DataFrame()
N = kn_te_pred.shape[0]
df2['ID'] = df1['ID'].loc(axis=0)[:N]
df2['item_cnt_month'] = kn_te_pred[214200]
# df2['item_cnt_month'] = df2.item_cnt_month.abs().iloc[214200:]
df2.to_csv('./result.csv', index= False)

# Hey Kagglers, Please upvote my notebook if you found it useful. It will help me be motivated and create many more simple notebooks in Kaggle competitions.